In [1]:
from rdflib import Graph, RDF, Namespace, Literal, URIRef
from SPARQLWrapper import SPARQLWrapper, JSON




In [2]:
def namesPlusID():
    """Deze funtie is voor het ophalen van alle namen die waaruit de gebruiker kan kiezen met hun bijbehorend ID en deze worden vervolgens opgeslagen in een dictionary """

    sparql = SPARQLWrapper("http://localhost:5820/people/query")
    sparql.setQuery("""
    PREFIX IMDB: <http://data.linkedmdb.org/resource/> 
    PREFIX film: <http://data.linkedmdb.org/resource/film/> 
    PREFIX foaf: <http://xmlns.com/foaf/0.1/> 
    PREFIX movie: <http://data.linkedmdb.org/resource/movie/> 
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
    PREFIX xml: <http://www.w3.org/XML/1998/namespace> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#> 

    SELECT ?id ?name  WHERE {
            ?id foaf:name ?name
            }

    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    people = {}
    for row in results["results"]["bindings"]:
        people[row["name"]["value"]] = row["id"]["value"]
    return people

#test
#namesPlusID()

In [3]:
def namesOnly(peopledict): 
    """Alleen de namen en dat in de juiste format zo dat ze op een mooie manier neergezet kunnen worden voor de gebruiker"""
    names = []
    for key, value in peopledict.iteritems():
        names.append(key[35:].replace("_"," "))
    return list(set(names))
    

#### Now we are going to write a couple of functions that calculate the shortest paths

In [4]:
def ConnectionsPerson(personName):
    personName = personName[35::]
    """This function uses the ID of a person to find all the people this person knows within the 
    local database."""
    #With this query we select all the names of the people the personID has workedWith
    sparql = SPARQLWrapper("http://localhost:5820/people/query")
    sparql.setQuery("""
    PREFIX IMDB: <http://data.linkedmdb.org/resource/> 
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>

    SELECT ?startName ?ID ?endName WHERE {{
      ?startID foaf:name IMDB:{val} .
      ?startID IMDB:workedWith ?ID .
      ?ID foaf:name ?endName .
    }}
    """.format(val=personName))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    resultDict = {}
    
    for person in results["results"]["bindings"]:
        resultDict[person['endName']['value']] = person['ID']['value']
        
    return resultDict

In [5]:
def desiredOutcome(inner_list):
    """This function makes sure the output of the function names findShortestPaths is printed
    in a correct way!"""
    for value in inner_list:
        one_path = []
        for person in value:
            one_path.append(person[35::])
        print " -> ".join(one_path)

In [6]:
def findShortestPaths(startName, targetName):
    #We have to make the startName into a list
    startNameTemp=startName.replace(" ","_")
    prefix = 'http://data.linkedmdb.org/resource/'
    startNameURI = prefix + startNameTemp
    #Now we created the NamesList we will start from
    NamesList = []
    NamesList.append([startNameURI])
    #Now we also need to make the targetName in URI readable format
    targetNameTemp=targetName.replace(" ","_")
    targetNameURI = prefix + targetNameTemp
    #We use x to see how many times we did a for loop so we can stop the script after a certain amount
    x = 0
    #We have to make the fill link of the name again to find the shortests paths
    correct_paths = []
    
    for name in NamesList:
        x+=1
        #My computer does roughly 165 loops every second. 4000 loops is roughly 20 seconds
        if x == 3000:
            if len(correct_paths) == 0:
                return 'We could not find any paths after calculating 3000 possible paths!'
            else:
                return correct_paths
        #If the last name in the path is the target person we add the path to the list.
        if name[-1] == targetNameURI:
            #We stop the function if we start finding longer paths
            if len(correct_paths) == 0:
                correct_paths.append(name)
            elif len(name) > len(correct_paths[-1]):
                return correct_paths
            else:
                correct_paths.append(name)
            #Making sure we stop the path if the last person is the one we are looking for.
            continue
        else:
            #Here we create a list of all the person the last person in the path knows
            new_steps = ConnectionsPerson(name[-1]).keys()
            #For every person this person knows, we create a new path.
            for person in new_steps:
                new_path = []
                #We make sure the names that were already in the path are also added to the new path. 
                if person in name:
                    continue
                else:
                    for value in name:
                        new_path.append(value)
                    #Now we add the new name we just found to the path
                    new_path.append(person)
                    #Since we add the new path to the names list, the function will keep on running!
                    NamesList.append(new_path)
    return "No paths were found!"

%time findShortestPaths('Jacqueline White', 'Season Hubley')

Wall time: 9.69 s


'We could not find any paths after calculating 3000 possible paths!'

In [ ]:
namesiddict = namesPlusID()
names = namesOnly(namesiddict)
names.sort()
people = namesPlusID()

In [ ]:
import wx 
from wx.lib.pubsub import pub 
import wx.richtext as rt
import itertools
import wx.lib.agw.hyperlink as hl
import re

class Mywin(wx.Frame): 
    def __init__(self, title): 
        wx.Frame.__init__(self, None, title=title, size=(500,200))
        panel = wx.Panel(self, wx.ID_ANY) 
        box = wx.BoxSizer(wx.VERTICAL)
        self.label = wx.StaticText(panel,label = "Your choices:" ,style = wx.ALIGN_CENTRE) 
        box.Add(self.label, 0 , wx.ALIGN_CENTER_HORIZONTAL |wx.ALL) 
        box.AddStretchSpacer( prop=3 )
        cblbl = wx.StaticText(panel,label = "You",style = wx.ALIGN_CENTRE| wx.ALIGN_TOP ) 
        box.Add(cblbl,0,wx.ALIGN_CENTER_HORIZONTAL|wx.ALL,5) 
        people = names
        #keuzes voor de gebruiker
        self.choice1 = wx.Choice(panel,choices = people) 
        box.Add(self.choice1,1,wx.EXPAND|wx.ALIGN_CENTER_HORIZONTAL|wx.ALL,5)
        #keuzes voor het target
        chlbl = wx.StaticText(panel,label = "Target",style = wx.ALIGN_CENTRE) 
        box.Add(chlbl,0,wx.EXPAND|wx.ALIGN_CENTER_HORIZONTAL|wx.ALL,5) 
        self.choice2 = wx.Choice(panel,choices = people) 
        box.Add(self.choice2,1,wx.EXPAND|wx.ALIGN_CENTER_HORIZONTAL|wx.ALL,5)
        #button definieeren
        self.btn = wx.Button(panel, -1, "Find connection")
        box.Add(self.btn, 0, wx.ALIGN_RIGHT, 10, 10)
        #bind de click actie aan de knop
        self.btn.Bind(wx.EVT_BUTTON, self.OnClicked) 
        panel.SetSizer(box) 
        self.Centre() 
        self.Show() 

        
        
    def OnClicked(self, event):
        """Showt het nieuwe scherm en geeft de tuple met gebruikersnaam en target naam mee"""
        SourceTarget = (names[self.choice1.GetCurrentSelection()],names[self.choice2.GetCurrentSelection()]) 
        secframe = SecondFrame(SourceTarget)
        
        secframe.Show()


class SecondFrame(wx.Frame):

    def __init__(self, parent):
        wx.Frame.__init__(self, None, title="Result", pos=(700,500), size=(600,350))
        panel = wx.Panel(self, wx.ID_ANY)
        #parent is de tuple met de gebruikersnaam en targetnaam
        self.parent = parent
        self.names = ["http://data.linkedmdb.org/resource/"+x.replace(" ","_") for x in parent]
        self.peopleid = [people[x] for x in self.names]
        box = wx.BoxSizer(wx.VERTICAL)
        

        shortestPaths = findShortestPaths(self.parent[0], self.parent[1])
        #chech of er connecties zijn als dat niet het geval is print dan, anders moet de code voor het presenteren van de resultaten worden gedraaid
        if shortestPaths == 'We could not find any paths after calculating 3000 possible paths!':
            lbl2 = wx.StaticText(panel,-1, style = wx.ALIGN_LEFT)
            lbl2.SetLabel(shortestPaths)
            box.Add(lbl2,0,wx.ALIGN_LEFT)
        else:
            toplabel = 'The easiest way for %s to get in contact with %s is through: \n' % (self.parent[0], self.parent[1])
            lbl = wx.StaticText(panel,-1,style = wx.ALIGN_CENTER)
            lblwrap = wx.StaticText(panel,-1,style = wx.ALIGN_LEFT) 
            lblwrap.SetLabel(toplabel)
            box.Add(lblwrap,0,wx.ALIGN_LEFT)
            peopleInPaths = itertools.chain.from_iterable(shortestPaths)
            peopleInPaths = list(set(peopleInPaths))
            peopleInPathsIdDict = {}
            for personName in peopleInPaths:
                try: 
                    peopleInPathsIdDict[personName] = namesiddict[personName]
                except KeyError:
                    pass
                
            #mooie string maken voor het weergeven van de connecties
            for path in shortestPaths:
                NameInText = [x[35:].replace('_', ' ') for x in path]
                pathlabel = str()
                for name in NameInText:
                    pathlabel = pathlabel+name+" -> "
                if pathlabel[-4:] == " -> ":
                    pathlabel = pathlabel[:-4]+' \n'
                lbl1 = wx.StaticText(panel,-1, style = wx.ALIGN_LEFT)
                lbl1.SetLabel(pathlabel)
                box.Add(lbl1,0,wx.ALIGN_LEFT)
            panel.SetSizer(box)
            #ophalen of iemand acteur regisseur of schrijver is en vervolgens een hyperlink maken naar zijn of haar linkedmdb
            for key, value in peopleInPathsIdDict.iteritems():
                m = re.findall('actor|director|writer', value)
                hyper1 = hl.HyperLinkCtrl(panel, -1, m[0]+': '+key[35:].replace('_', ' '),URL=value)
                box.Add(hyper1,0,wx.ALIGN_LEFT)
        
app = wx.App() 
Mywin('test') 
#app draaien
app.MainLoop()